#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CLIENT CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter DV360 User Audit Parameters
Gives DV clients ability to see which users have access to which parts of an account. Loads DV user profile mappings using the API into BigQuery and connects to a DataStudio dashboard.
 1. DV360 only permits SERVICE accounts to access the user list API endpoint, be sure to provide and permission one.
 1. Wait for <b>BigQuery->->->DV_*</b> to be created.
 1. Wait for <b>BigQuery->->->Barnacle_*</b> to be created, then copy and connect the following data sources.
 1. Join the <a href='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/9f6b9e62-43ec-4027-849a-287e9c1911bd' target='_blank'>Barnacle DV Report</a>.
 1. Click Edit->Resource->Manage added data sources, then edit each connection to connect to your new tables above.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for writing data.
  'auth_write': 'service',  # Credentials used for writing data.
  'partner': '',  # Partner ID to run user audit on.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute DV360 User Audit
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import Configuration
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'doubleclickbidmanager',
      'version': 'v1.1',
      'function': 'queries.listqueries',
      'alias': 'list',
      'results': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'DV_Reports'
        }
      }
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'displayvideo',
      'version': 'v1',
      'function': 'partners.list',
      'kwargs': {
        'fields': 'partners.displayName,partners.partnerId,nextPageToken'
      },
      'results': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'DV_Partners'
        }
      }
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'displayvideo',
      'version': 'v1',
      'function': 'advertisers.list',
      'kwargs': {
        'partnerId': {'field': {'name': 'partner','kind': 'integer','order': 2,'default': '','description': 'Partner ID to run user audit on.'}},
        'fields': 'advertisers.displayName,advertisers.advertiserId,nextPageToken'
      },
      'results': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'DV_Advertisers'
        }
      }
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'displayvideo',
      'version': 'v1',
      'function': 'users.list',
      'kwargs': {
      },
      'results': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'DV_Users'
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': "SELECT           U.userId,           U.name,           U.email,           U.displayName,           REGEXP_EXTRACT(U.email, r'@(.+)') AS Domain,           IF (ENDS_WITH(U.email, '.gserviceaccount.com'), 'Service', 'User') AS Authentication,           IF((Select COUNT(advertiserId) from UNNEST(U.assignedUserRoles)) = 0, 'Partner', 'Advertiser') AS Scope,           STRUCT(             AUR.partnerId,             P.displayName AS partnerName,             AUR.userRole,             AUR.advertiserId,             A.displayName AS advertiserName,             AUR.assignedUserRoleId           ) AS assignedUserRoles,           FROM `{dataset}.DV_Users` AS U,           UNNEST(assignedUserRoles) AS AUR           LEFT JOIN `{dataset}.DV_Partners` AS P           ON AUR.partnerId=P.partnerId           LEFT JOIN `{dataset}.DV_Advertisers` AS A           ON AUR.advertiserId=A.advertiserId         ",
        'parameters': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'view': 'Barnacle_User_Roles'
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': "SELECT           R.*,           P.displayName AS partnerName,           A.displayName AS advertiserName,           FROM (           SELECT             queryId,             (SELECT CAST(value AS INT64) FROM UNNEST(R.params.filters) WHERE type = 'FILTER_PARTNER' LIMIT 1) AS partnerId,             (SELECT CAST(value AS INT64) FROM UNNEST(R.params.filters) WHERE type = 'FILTER_ADVERTISER' LIMIT 1) AS advertiserId,             R.schedule.frequency,             R.params.metrics,             R.params.type,             R.metadata.dataRange,             R.metadata.sendNotification,             DATE(TIMESTAMP_MILLIS(R.metadata.latestReportRunTimeMS)) AS latestReportRunTime,           FROM `{dataset}.DV_Reports` AS R) AS R           LEFT JOIN `{dataset}.DV_Partners` AS P           ON R.partnerId=P.partnerId           LEFT JOIN `{dataset}.DV_Advertisers` AS A           ON R.advertiserId=A.advertiserId         ",
        'parameters': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'view': 'Barnacle_Reports'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(Configuration(project=CLOUD_PROJECT, client=CLIENT_CREDENTIALS, user=USER_CREDENTIALS, verbose=True), TASKS, force=True)
